In [ ]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

class Simulation:
    def generate_interarrival(self):  
        ### Generate exponentially distributed arrival Time
        return  (-1*self.arrival_time*math.log(random.random()))  

    def generate_service(self):        
        ### Generate exponentially distributed service Time
        return  (-1*self.service_time*math.log(random.random())) 
    
    def initialize(self):                    
        ### Initialize the simulation objects 
        
        self.server_status      = [0 for i in range(self.no_of_servers)]
        self.area_server_status = [0 for i in range(self.no_of_servers)]
        self.server_utilization = [0 for i in range(self.no_of_servers)]
        self.time_next_event    = [0 for i in range(self.no_of_servers + 1)]
                
        self.clock_time = 0.0        
        self.time_last_event = 0.0
        self.num_events = self.no_of_servers + 1
        
        self.lost_calls = 0
        self.total_arrivals = 0
        self.total_departed = 0
        self.blocking_probability = 0.0
        self.total_server_utilization = 0.0
        
        for i in range(len(self.time_next_event)):
            if i == 0:
                self.time_next_event[i] = self.generate_interarrival()
            else:
                self.time_next_event[i] = float('inf')

    def timing(self):
        ### Advances the time for simulation and calculate the next event type
        
        self.min_time_next_event = float('inf');

        ## Determine the event type of the next event to occur
        for i in range(self.num_events):
            if (self.time_next_event[i] <= self.min_time_next_event):
                self.min_time_next_event = self.time_next_event[i]
                self.next_event_type = i

        self.time_last_event = self.clock_time
        
        ## advance the simulation clock
        self.clock_time = self.time_next_event[self.next_event_type]

    def arrive(self):
        ### Schedule next call arrival, also check if any server is idle for providing service,
        ### if not, then call is lost.
        
        self.time_next_event[0] = self.clock_time + self.generate_interarrival() 
        self.total_arrivals += 1
        
        ## Find out the idle server
        i = 0
        server_idle = -1
    
        while (server_idle == -1 and i < self.num_events - 1):
            if (self.server_status[i] == 0):
                server_idle = i  ## Found Idle server
                break
            i += 1

        if server_idle >= 0:  ## If any server is Idle
            self.server_status[server_idle] = 1
            self.time_next_event[server_idle + 1] = self.clock_time +  self.generate_service()            
        else:  ## else all servers are BUSY, call is lost
            self.lost_calls += 1
        
    def depart(self, j):
        ### Departure of the call
        
        self.server_status[j-1] = 0
        self.time_next_event[j] = float('inf')
        self.total_departed += 1

    def update_time_avg_stats(self):        
        ### Update area accumulators for time-average statistics
        
        self.time_past = self.clock_time - self.time_last_event

        for i in range(self.no_of_servers):
            self.area_server_status[i] += self.time_past * self.server_status[i]

    def report(self):
        ### Compute the desired measures of performance, 
        ### i.e Simulation's server utilization and call blocking probability
        
        self.blocking_probability = self.lost_calls/self.num_calls_required ### Calculate Simulation's call blocking probability

        for i in range(self.no_of_servers):
            self.server_utilization[i] = self.area_server_status[i] / self.clock_time
            self.total_server_utilization += self.area_server_status[i]

        self.total_server_utilization = self.total_server_utilization /self.clock_time/ self.no_of_servers
        
    def main(self, trial):
        ### Main method to start the simulation
        
        self.no_of_servers          = 16      ## Simulation for 16 servers
        self.num_calls_required     = 100000  ## Simulation with 100000 calls
        self.service_time           = 100     ## 100 Secs between two services
        
        if trial == 0:
            self.arrival_time       = 10      ## 10 Secs between two arrivals
        elif trial == 1:
            self.arrival_time       = 11.5    ## 11.5 Secs between two arrivals
        else:
            self.arrival_time       = 100     ## 100 Secs between two arrivals
        
        self.initialize(); 

        while (self.total_arrivals < self.num_calls_required):
            self.timing()  

            if (self.next_event_type == 0):
                self.arrive()  
            else:
                self.depart(self.next_event_type)
                
            self.update_time_avg_stats()
            
        self.report();

for x in range(3):      # Two trials with different arrival rate, just to plot and compare the performance
    
    np.random.seed(0)
    s = Simulation()
    s.main(x)

    lamda = 1/s.arrival_time
    mu    = 1/s.service_time
    c     = s.no_of_servers
    
    ### Calculate math call blocking probability
    numerator_for_Pc   = 0.0
    denominator_for_Pc = 0.0
    math_blocking_prob = 0.0 
    
    numerator_for_Pc   = ((lamda/mu)**c)/math.factorial(c)
    
    for i in range(c + 1):
        denominator_for_Pc +=  (((lamda/mu)**i)/math.factorial(i))
    
    math_blocking_prob = numerator_for_Pc/denominator_for_Pc 
    

    print("*******************Arrival rate = " +str(1/s.arrival_time) + "**********************")
    print("Service Rate                      = " + str(1/s.service_time))
    print("Number of Calls                   = " + str(s.num_calls_required))
    print("Simulation Blocking Probability   = " + str(s.blocking_probability))
    print("Math Blocking Probability         = " + str(math_blocking_prob))
    print("Servers Utilization               = " + str(s.total_server_utilization))
    print("Arrivals(with loss)               = " + str(s.total_arrivals))
    print("Arrivals successful               = " + str(s.total_arrivals - s.lost_calls))
    print("Departures                        = " + str(s.total_departed + sum(s.server_status)))
    print("Lost Arrivals                     = " + str(s.lost_calls))
    print("\n")

    num_of_servers  = [i+1 for i in range(s.no_of_servers)]
    plt.scatter(num_of_servers, s.server_utilization, label = 'Arrival Time - '+str(s.arrival_time))
plt.legend()
plt.title('Server Utilization vs No. of Servers')
plt.xlabel('No. of Servers')
plt.ylabel('Server Utilization')
plt.show() 